<a href="https://colab.research.google.com/github/keywoong/deeplearning_with_python/blob/main/3_4_Getting_started_with_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3.6 Predicting house prices : a regression example
#### 회귀 문제는 라벨로 구별하는 것이 아니라 값을 예측하는 문제이다. regression과 달리 logistic regression은 분류문제에 사용되는 알고리즘이다.
## 3.6.1 The Boston Housing Price dataset

In [ ]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

print(train_data.shape)
print(test_data.shape)

(404, 13)
(102, 13)


#### 위의 결과처럼, training 데이터셋은 404개이고, test 데이터셋은 102개이다. 그리고 그들을 구성하는 특징의 개수는 총 13개이다. (특징은 범죄율, 평당 가격, 고속도로 접근 가능성 등이 있겠다.)
#### target값은 집값이다. (단위는 천달러)

In [ ]:
print(train_targets)

[15.2 42.3 50.  21.1 17.7 18.5 11.3 15.6 15.6 14.4 12.1 17.9 23.1 19.9
 15.7  8.8 50.  22.5 24.1 27.5 10.9 30.8 32.9 24.  18.5 13.3 22.9 34.7
 16.6 17.5 22.3 16.1 14.9 23.1 34.9 25.  13.9 13.1 20.4 20.  15.2 24.7
 22.2 16.7 12.7 15.6 18.4 21.  30.1 15.1 18.7  9.6 31.5 24.8 19.1 22.
 14.5 11.  32.  29.4 20.3 24.4 14.6 19.5 14.1 14.3 15.6 10.5  6.3 19.3
 19.3 13.4 36.4 17.8 13.5 16.5  8.3 14.3 16.  13.4 28.6 43.5 20.2 22.
 23.  20.7 12.5 48.5 14.6 13.4 23.7 50.  21.7 39.8 38.7 22.2 34.9 22.5
 31.1 28.7 46.  41.7 21.  26.6 15.  24.4 13.3 21.2 11.7 21.7 19.4 50.
 22.8 19.7 24.7 36.2 14.2 18.9 18.3 20.6 24.6 18.2  8.7 44.  10.4 13.2
 21.2 37.  30.7 22.9 20.  19.3 31.7 32.  23.1 18.8 10.9 50.  19.6  5.
 14.4 19.8 13.8 19.6 23.9 24.5 25.  19.9 17.2 24.6 13.5 26.6 21.4 11.9
 22.6 19.6  8.5 23.7 23.1 22.4 20.5 23.6 18.4 35.2 23.1 27.9 20.6 23.7
 28.  13.6 27.1 23.6 20.6 18.2 21.7 17.1  8.4 25.3 13.8 22.2 18.4 20.7
 31.6 30.5 20.3  8.8 19.2 19.4 23.1 23.  14.8 48.8 22.6 33.4 21.1 13.6
 32.2 13.1

## 3.6.2 Preparing the data
#### 데이터들은 10000달러에서 50000달러 사이에 분포하는데 이는 widespreading하다. 우리는 이 데이터들을 정규화를 시켜서 0을 기준으로 분포하도록 만들어 줘야 한다.

In [ ]:
# Normaliziing the data
mean = train_data.mean(axis = 0)
train_data -=mean
std = train_data.std(axis = 0)
train_data /= std

test_data -= mean
test_data /= std

print(train_data)

[[-0.27224633 -0.48361547 -0.43576161 ...  1.14850044  0.44807713
   0.8252202 ]
 [-0.40342651  2.99178419 -1.33391162 ... -1.71818909  0.43190599
  -1.32920239]
 [ 0.1249402  -0.48361547  1.0283258  ...  0.78447637  0.22061726
  -1.30850006]
 ...
 [-0.40202987  0.99079651 -0.7415148  ... -0.71712291  0.07943894
  -0.67776904]
 [-0.17292018 -0.48361547  1.24588095 ... -1.71818909 -0.98764362
   0.42083466]
 [-0.40422614  2.04394792 -1.20161456 ... -1.30866202  0.23317118
  -1.15392266]]


#### 위의 과정처럼 정규화 과정을 진행한다.
#### 정규화 : $ Z = (X-m) / \sigma $

## 3.6.3 Building your network
#### 404개의 적은 데이터만 사용이 가능하므로, 64개의 unit을 가지는 2개의 은닉층으로 구성된 매우 작은 네트워크를 만들 것이다.
#### 일반적으로 데이터의 개수가 적으면 오버피팅이 발생할 가능성이 높아지기 때문에, 네트워크를 작게 만드는 것이 하나의 해결방법이 될 수 있다.

In [ ]:
from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation = 'relu', input_shape = (train_data.shape[1],)))
    model.add(layers.Dense(64, activation = 'relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer = 'rmsprop', loss = 'mse', metrics = ['mae'])
    return model

#### 이 모델의 마지막 층은 1개의 unit만이 있고, 활성화 함수도 없는 것으로 보아 스칼라를 출력하는 선형함수임을 알 수 있다.
#### 이 선형함수를 통해 원하는 값을 예측할 수 있는 것이다 (regression)
#### 만약 sigmoid함수를 사용했다면, 0과 1 사이의 값으로 출력하므로 순수 원하는 값을 얻을 수 없다.
#### 이 네트워크에서는 `mse` 손실함수를 사용했다. `(mean squared error)` 최소제곱법으로, 회귀 문제에 자주 사용된다. (2차 norm)
#### 또한 `mae`를 monitoring metric으로 선정했다. `(Mean Absolute Error)` 이것은 예측값과 실제값의 절대적인 차이값이다. 

## 3.6.4 Validating your approach using K-fold validation
#### 이진분류나 다중분류때 했던 것처럼 training set으로 매개변수를 수정해나가고, 마지막으로 validation data로 점검을 하려고 한다.
#### 그런데 이 dataset은 자료의 수가 너무 적기 때문에 (404개, 102개) 그렇게 하기가 힘들다.
#### 이러한 경우에 K-fold를 사용한다.
#### 사용 가능한 데이터를 K 개의 파티션으로 분할하고, K 개의 동일한 모델을 인스턴스화하고, 나머지 파티션에서 평가하는 동안 K-1 파티션에서 각 모델을 훈련하는 것으로 구성됩니다.
#### 사용 된 모델의 검증 점수는 획득 한 K 검증 점수의 평균입니다.

In [ ]:
# K-fold Validation
import numpy as np

k = 4
num_val_samples = int(len(train_data) / k) # k개의 파티션으로 분할
num_epochs = 100
all_scores = []

for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i*num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples : (i + 1) * num_val_samples]
    
    partial_train_data = np.concatenate(
    [train_data[:i * num_val_samples],
    train_data[(i + 1) * num_val_samples :]],
    axis = 0)
    partial_train_targets = np.concatenate(
    [train_targets[:i * num_val_samples],
    train_targets[(i + 1) * num_val_samples :]],
    axis = 0)
    
    model = build_model()
    model.fit(partial_train_data, partial_train_targets, epochs = num_epochs, batch_size = 1, verbose = 0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose = 0)
    all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [ ]:
print(all_scores)
print('----------------')
print(np.mean(all_scores))

[2.1120738983154297, 2.3811111450195312, 2.6254844665527344, 2.4106967449188232]
----------------
2.3823415637016296


#### all_scores를 보면 2.1에서 2.7까지 다른 validation score가 나온것을 알 수 있다.
#### 평균은 약 2.4

In [ ]:
# Saving the validation logs at each fold
num_epochs = 500
all_mae_histories = []
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i*num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples : (i + 1) * num_val_samples]
    
    partial_train_data = np.concatenate(
    [train_data[:i * num_val_samples],
    train_data[(i + 1) * num_val_samples :]],
    axis = 0)
    partial_train_targets = np.concatenate(
    [train_targets[:i * num_val_samples],
    train_targets[(i + 1) * num_val_samples :]],
    axis = 0)
    
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets, validation_data = (val_data, val_targets), epochs = num_epochs, batch_size = 1, verbose = 0)
    mae_history = history.history['val_mean_absolute_error']
    all_mae_histories.append(mae_history)

processing fold # 0


KeyError: 'val_mean_absolute_error'